<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 813ms/step - loss: 0.6790 - accuracy: 0.5803 - val_loss: 0.6714 - val_accuracy: 0.5659
Epoch 2/10
26/26 [==============================] - 20s 777ms/step - loss: 0.6498 - accuracy: 0.6436 - val_loss: 0.6436 - val_accuracy: 0.5951
Epoch 3/10
26/26 [==============================] - 20s 773ms/step - loss: 0.6006 - accuracy: 0.7117 - val_loss: 0.6412 - val_accuracy: 0.5317
Epoch 4/10
26/26 [==============================] - 20s 777ms/step - loss: 0.5532 - accuracy: 0.7567 - val_loss: 0.5069 - val_accuracy: 0.7902
Epoch 5/10
26/26 [==============================] - 20s 769ms/step - loss: 0.5114 - accuracy: 0.7762 - val_loss: 0.5601 - val_accuracy: 0.6732
Epoch 6/10
26/26 [==============================] - 20s 766ms/step - loss: 0.4333 - accuracy: 0.8175 - val_loss: 0.3832 - val_accuracy: 0.8537
Epoch 7/10
26/26 [==============================] - 20s 765ms/step - loss: 0.3886 - accuracy: 0.8382 - val_loss: 0.6404 - val_accuracy: 0.6390

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.64.h5
26/26 - 21s - loss: 0.6652 - accuracy: 0.6168 - val_loss: 0.6422 - val_accuracy: 0.7171
Epoch 2/5

Epoch 00002: saving model to weights.02-0.57.h5
26/26 - 20s - loss: 0.6080 - accuracy: 0.7360 - val_loss: 0.5745 - val_accuracy: 0.7561
Epoch 3/5

Epoch 00003: saving model to weights.03-0.61.h5
26/26 - 20s - loss: 0.5456 - accuracy: 0.7555 - val_loss: 0.6073 - val_accuracy: 0.6341
Epoch 4/5

Epoch 00004: saving model to weights.04-0.44.h5
26/26 - 20s - loss: 0.4680 - accuracy: 0.8139 - val_loss: 0.4418 - val_accuracy: 0.8439
Epoch 5/5

Epoch 00005: saving model to weights.05-0.36.h5
26/26 - 20s - loss: 0.4134 - accuracy: 0.8309 - val_loss: 0.3598 - val_accuracy: 0.8683


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6617 - accuracy: 0.6010 - val_loss: 0.6377 - val_accuracy: 0.6537


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6681 - accuracy: 0.5779 - val_loss: 0.6665 - val_accuracy: 0.4976
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 21s - loss: 0.6268 - accuracy: 0.6375 - val_loss: 0.5944 - val_accuracy: 0.8049


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6618 - accuracy: 0.6338 - val_loss: 0.6301 - val_accuracy: 0.7073
Epoch 2/50
26/26 - 20s - loss: 0.6054 - accuracy: 0.6946 - val_loss: 0.5742 - val_accuracy: 0.7268
Epoch 3/50
26/26 - 20s - loss: 0.5398 - accuracy: 0.7433 - val_loss: 0.5271 - val_accuracy: 0.7122
Epoch 4/50
26/26 - 20s - loss: 0.4982 - accuracy: 0.7470 - val_loss: 0.4809 - val_accuracy: 0.7415
Epoch 5/50
26/26 - 20s - loss: 0.4745 - accuracy: 0.7603 - val_loss: 0.4494 - val_accuracy: 0.7659
Epoch 6/50
26/26 - 20s - loss: 0.4594 - accuracy: 0.7701 - val_loss: 0.4107 - val_accuracy: 0.8244
Epoch 7/50
26/26 - 20s - loss: 0.3973 - accuracy: 0.8200 - val_loss: 0.4101 - val_accuracy: 0.8293
Epoch 8/50
26/26 - 20s - loss: 0.3589 - accuracy: 0.8467 - val_loss: 0.3730 - val_accuracy: 0.8390
Epoch 9/50
26/26 - 20s - loss: 0.3134 - accuracy: 0.8783 - val_loss: 0.2653 - val_accuracy: 0.9171
Epoch 10/50
26/26 - 20s - loss: 0.2624 - accuracy: 0.9100 - val_loss: 0.2013 - val_accuracy: 0.9415
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 781ms/step - loss: 0.6719 - accuracy: 0.5462 - val_loss: 0.6606 - val_accuracy: 0.5805
Epoch 2/5
26/26 [==============================] - 19s 750ms/step - loss: 0.6355 - accuracy: 0.6582 - val_loss: 0.6280 - val_accuracy: 0.6341
Epoch 3/5
26/26 [==============================] - 20s 769ms/step - loss: 0.5953 - accuracy: 0.7117 - val_loss: 0.5981 - val_accuracy: 0.6488
Epoch 4/5
26/26 [==============================] - 20s 758ms/step - loss: 0.5374 - accuracy: 0.7530 - val_loss: 0.7519 - val_accuracy: 0.4878
Epoch 5/5
26/26 [==============================] - 19s 746ms/step - loss: 0.5184 - accuracy: 0.7518 - val_loss: 0.4517 - val_accuracy: 0.8049


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.546229,0.672417,0.580488,0.660583
1,1,0.658151,0.635923,0.634146,0.627976
2,2,0.711679,0.596046,0.648780,0.598070
3,3,0.753041,0.537388,0.487805,0.751857
4,4,0.751825,0.518961,0.804878,0.451669


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 20s 778ms/step - loss: 0.6779 - accuracy: 0.5584 - val_loss: 0.6975 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 31s 1s/step - loss: 0.6517 - accuracy: 0.5803 - val_loss: 0.6544 - val_accuracy: 0.6585

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 800ms/step - loss: 0.6366 - accuracy: 0.7141 - val_loss: 0.6499 - val_accuracy: 0.6341

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6295 - accuracy: 0.6959 - val_loss: 0.6436 - val_accuracy: 0.6829

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6258

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 23s 871ms/step - loss: 0.6595 - accuracy: 0.6302 - val_loss: 0.6417 - val_accuracy: 0.6732
Epoch 2/50
26/26 [==============================] - 20s 783ms/step - loss: 0.5897 - accuracy: 0.7275 - val_loss: 0.5537 - val_accuracy: 0.7268
Epoch 3/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5250 - accuracy: 0.7437
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 785ms/step - loss: 0.5245 - accuracy: 0.7433 - val_loss: 0.5570 - val_accuracy: 0.7122
Epoch 4/50
26/26 [==============================] - 20s 769ms/step - loss: 0.4682 - accuracy: 0.7762 - val_loss: 0.4649 - val_accuracy: 0.7561
Epoch 5/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4410 - accuracy: 0.7825
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 20s 756ms/step - loss: 0.4402 - accuracy: 0.7822 - val_loss: 0.472

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 5119), started 0:17:27 ago. (Use '!kill 5119' to kill it.)